In [ ]:
import pandas as pd

# downloaded in batch from https://asmith.ucdavis.edu/data/prism-weather
# monthly from 2019-2023
csv_file_path = "monthly_avg_temp_by_zip.csv"

# Load the CSV into a pandas DataFrame
df = pd.read_csv(csv_file_path)

df[df['zipcode'] == 32601]

In [ ]:
# Group by the 'zipcode' column and calculate the average of the 'tavg' column
# Assuming the 'tavg' column contains numerical data
zipcode_avg_tavg = df.groupby('zipcode')['tavg'].mean().reset_index()
zipcode_avg_tavg.columns = ['zip_code', 'average_monthly_temp']

# Display the result, average temp for every zip in FL
print(zipcode_avg_tavg)

In [ ]:
import geopandas as gpd

# MDC zip code polygons
# https://gis-mdc.opendata.arcgis.com/datasets/MDC::zip-code/about
# downloaded 9/20
shapefile_path = 'mdc_polygons/Zip_Code.shp'
gdf_shapefile = gpd.read_file(shapefile_path)

gdf = gdf_shapefile.rename(columns={'ZIPCODE': 'zip_code'})
gdf['zip_code'] = gdf['zip_code'].astype(int)

gdf_shapefile.head()


In [ ]:

# Read a GeoJSON file containing US state boundaries
# from 2018
# https://geohub-bcgis.opendata.arcgis.com/datasets/BCGIS::zip-codes/about
broward_gdf = gpd.read_file('broward_zips.geojson')
broward_gdf = gdf.rename(columns={'ZIP_CODE': 'zip_code'})
broward_gdf['zip_code'] = broward_gdf['zip_code'].astype(int)
# 00041 is dropped, don't believe it is a valid zip code
broward_gdf = broward_gdf.drop(broward_gdf.index[0])

broward_gdf.head()

In [ ]:
# Concatenate the DataFrames
gdf_concat = pd.concat([broward_gdf, gdf], ignore_index=True)

gdf_concat

In [ ]:
# Assuming 'ZIPCODE' is the common column between shapefile and CSV
merged_gdf = gdf_concat.merge(zipcode_avg_tavg, on='zip_code')

merged_gdf['average_monthly_temp_f'] = (merged_gdf['average_monthly_temp'] * 9/5) + 32

# View the merged data
merged_gdf.head()

In [ ]:
# Output the DataFrame as a SHP file
merged_gdf.to_file('avg_temp_by_zip_with_mdc_broward_polygons.shp', index=False)

# Plot the geometries with ZIP codes as color
merged_gdf.plot(column='zip_code', legend=True)

In [ ]:
import matplotlib.pyplot as plt

# Assuming merged_gdf is your merged GeoDataFrame
# Set the figure size
plt.figure(figsize=(12, 10))

# Plot the GeoDataFrame
# Use 'average_monthly_temp' for coloring the polygons
merged_gdf.plot(column='average_monthly_temp',
                cmap='OrRd',  # You can change the colormap if desired
                linewidth=0.8,
                edgecolor='black',
                legend=True)

# Add titles and labels
plt.title('Average Monthly Temperature by ZIP Code')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.show()

In [ ]:
merged_gdf.loc[0, 'geometry']

In [ ]:
# Assuming merged_gdf is already defined and is a GeoPandas DataFrame
# Ensure merged_gdf has a column 'average_monthly_temp_f' for the data to be visualized

# Convert GeoPandas DataFrame to GeoJSON format
geojson = merged_gdf.to_json()

# Debugging: Print a sample of the GeoJSON data
print("GeoJSON Sample:", geojson[:500])  # Print the first 500 characters

# Debugging: Print a sample of the DataFrame
print("DataFrame Sample:")
print(merged_gdf.head())

In [ ]:
# Check current CRS
print(merged_gdf.crs)

# If the CRS is not WGS84 (EPSG:4326), reproject it
if merged_gdf.crs != "EPSG:4326":
    merged_gdf = merged_gdf.to_crs(epsg=4326)